In [2]:
import requests
from goose3 import Goose
import spacy
import re

In [3]:
def sbd_component(doc):
    for i, token in enumerate(doc[:-2]):
        # define sentence start if period + titlecase token
        if token.text == '.' and doc[i+1].is_title:
            doc[i+1].is_sent_start = True
        elif token.text == '.' and doc[i+1].is_upper:
            doc[i+1].is_sent_start = True
        elif token.text == '.' and doc[i+1].is_punct:
            doc[i+1].is_sent_start = True
    return doc

def punct_check(text):
    str_in_doublequotes = r'"([^"]*)"'
    in_quotes = re.findall(str_in_doublequotes, text)
    for z in in_quotes:
        text = text.replace(z,z.replace('.','**y**'))
    return text

nlp = spacy.load("../model_postag_ner/")
nlp.add_pipe(sbd_component, before='parser')  # insert before the parser

USER_AGENT = ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6)'
              ' AppleWebKit/537.36 (KHTML, like Gecko)'
              ' Chrome/58.0.3029.96 Safari/537.36')

KNOWN_PATTERNS = [
            {
                'attr': 'class',
                'value': 'short-story'
            },
            {
                'attr': 'itemprop',
                'value': 'articleBody'
            },
            {
                'attr': 'id',
                'value': 'article-body'
            },
            {
                'attr': 'class',
                'value': 'article-body'
            },
            {
                'attr': 'class',
                'value': 'article-text'
            },
            {
                'attr': 'class',
                'value': 'post-content'
            },
            {
                'attr': 'class',
                'value': 'g-content'
            },
            {
                'tag': 'article'
            },
            {
                "attr": "class",
                "value": "content_detail"
            }
        ] 

lang = "id"

def getContent(url):
    g = Goose({
                'enable_image_fetching': False,
                'use_meta_language': False,
                'target_language': lang,
                'browser_user_agent': USER_AGENT,
                'known_context_patterns': KNOWN_PATTERNS,
            })
    article = g.extract(url=url)
     
    text = article.cleaned_text.replace(".",". ")
    text = text.replace("\n"," ")
    text = text.replace("  "," ")
    text = punct_check(text)
    doc = nlp(text)
    senteces = []
    for sent in doc.sents:
        txt = sent.text
        txt = txt.strip().replace("**y**",".")
        senteces.append(txt)
    sentences = "\n\n".join(senteces)
    return sentences



In [4]:
import feedparser

In [5]:
url_feed = """https://news.google.com/news/rss/headlines/section/topic/NATION.id_id/Indonesia?ned=id_id&hl=id&gl=ID"""
d = feedparser.parse(url_feed)

In [6]:
for i in d['entries'][:1]:
    print(i['title'])
    print(i['links'][0]['href'])
    url = i['links'][0]['href']
    content = getContent(url)
    print(content.replace(".",". "))
    print("\n")

Adik Ahok: Sidang Lagi ke PN Jakut, Kayak Deja Vu
https://www.cnnindonesia.com/nasional/20180130095343-20-272531/adik-ahok-sidang-lagi-ke-pn-jakut-kayak-deja-vu
Jakarta, CNN Indonesia -- Basuki Tjahaja Purnama alias Ahok akan menjalani sidang perdana gugatan cerai terhadap istrinya Veronica Tan, Rabu (31/1) besok. 

Sidang yang digelar di gedung Pengadilan Negeri Jakarta Utara (PN Jakut) itu, bagi adik Ahok, Fifi Lety Tjahaja Purnama seperti deja vu. 

"Tanggal 31 harus sidang lagi ke PN Jakut, ini kayak deja vu saja balik ke sidang tahun lalu," katanya di akun Instagramnya, Selasa (30/1). 

Dalam akun instagramnya itu, Fifi juga mengunggah foto saat mendampingi Ahok di Pengadilan Negeri Jakarta Utara. 

Fifi menjadi salah satu pengacara Ahok, yang didakwa menista agama. 

"Koko Ahok secara fisik di penjara, tetapi justru di penjara jiwanya bebas, Mungkin ada yang secara fisik bebas tetapi jiwanya justru terpenjara," tulis Fifi. 

Sebelumnya, Pengadilan Negeri Jakarta Utara akan mengge

In [8]:
print(getContent("http://nasional.kompas.com/read/2018/01/30/10414581/cerita-menegangkan-jokowi-ke-afghanistan-selimut-jadi-syal-hingga-sujud"))

JAKARTA, KOMPAS. com — Kenekatan Presiden Joko Widodo berkunjung ke Afghanistan yang tengah dalam kondisi tidak stabil menghadirkan kesan dan cerita tersendiri bagi para pembantunya yang ikut serta dalam rombongan.  Sekretaris Kabinet Pramono Anung menceritakan, Pemerintah Afghanistan memberlakukan pengamanan yang sangat ketat kepada Presiden Jokowi dan rombongan.  Pengamanan ketat mengawal delegasi setibanya di Bandara Internasional Hamid Karzai, Kabul, sampai Istana Agr tempat Presiden Afghanistan Ashraf Ghani sudah menunggu.  "Sepanjang jalan dari Airport ke Istana Presiden Afghanistan melalui jalan-jalan berbeton, kendaraan lapis baja dan 2 heli terbang diatas mobil Presiden," cerita Pramono lewat akun twitternya, @pramonoanung.  Dua hari sebelum kedatangan Presiden Jokowi, Kabul memang baru saja mengalami serangan teror bom bunuh diri.

Korbannya mencapai 103 orang.

Namun, serangan teror itu tak menyurutkan niat Jokowi berkunjung ke Afghanistan.  Rencana kunjungan ini memang suda